In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM,Qwen3ForCausalLM
# /share/public/public_models/Qwen3-14B/
# /share/others/public_models/Qwen3-14B/

tokenizer = AutoTokenizer.from_pretrained("/share/others/public_models/Qwen3-14B/")
# model = AutoModelForCausalLM.from_pretrained("/share/others/public_models/Qwen3-14B/")

/home/xujiaming/xujiaming/anaconda3/envs/specmod/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
messages = [
    {"role": "system",
        "content": "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."},
]
messages.append({
            "role": "user",
            "content": "How are you"
        })
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
)

In [14]:
inputs.input_ids.squeeze(0).tolist()


[151644,
 8948,
 198,
 2610,
 525,
 264,
 10950,
 11,
 48050,
 323,
 10745,
 17847,
 13,
 23240,
 4226,
 438,
 1492,
 3641,
 438,
 3204,
 11,
 1393,
 1660,
 6092,
 13,
 220,
 4615,
 11253,
 1265,
 537,
 2924,
 894,
 27756,
 11,
 88635,
 11,
 24207,
 11,
 63782,
 11,
 20836,
 11,
 11406,
 11,
 476,
 11816,
 2213,
 13,
 5209,
 5978,
 429,
 697,
 14507,
 525,
 39318,
 73215,
 323,
 6785,
 304,
 6993,
 382,
 2679,
 264,
 3405,
 1558,
 537,
 1281,
 894,
 5530,
 11,
 476,
 374,
 537,
 2097,
 1832,
 55787,
 11,
 10339,
 3170,
 4518,
 315,
 35764,
 2494,
 537,
 4396,
 13,
 1416,
 498,
 1513,
 944,
 1414,
 279,
 4226,
 311,
 264,
 3405,
 11,
 4486,
 1513,
 944,
 4332,
 895,
 1995,
 13,
 151645,
 198,
 151644,
 872,
 198,
 4340,
 525,
 498,
 151645,
 198,
 151644,
 77091,
 198]

In [15]:
from typing import Optional
import json
def load_questions(question_file: str, begin: Optional[int], end: Optional[int]):
    """Load questions from a file."""
    questions = []
    with open(question_file, "r") as ques_file:
        for line in ques_file:
            if line:
                questions.append(json.loads(line))
    questions = questions[begin:end]
    return questions

dataset_path = '/home/xujiaming/xujiaming/Paper/dataset/mt-bench/question.jsonl'
questions = load_questions(dataset_path,0,1)

In [18]:
questions[0]

{'question_id': 81,
 'category': 'writing',
 'turns': ['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
  'Rewrite your previous response. Start every sentence with the letter A.']}

In [ ]:
a = [[[] for i in range(10)] for _ in range(10)]

In [ ]:
a={}

In [2]:
a[100] = 10

In [3]:
a

{100: 10}

In [60]:
a[0][0].append(5)

In [63]:
a

[[[5], [5], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []]]

In [62]:
a[0][1] = a[0][0]

In [27]:
import torch
import torch.nn.functional as F

v1 = torch.tensor([1., 2., 3.]).unsqueeze(0).unsqueeze(0)
v2 = torch.tensor([1., 1., 3.]).unsqueeze(0).unsqueeze(0)

# 对于 1D 向量，我们需要在 dim=0 上计算
similarity_1d = F.cosine_similarity(v1, v2, dim= -1).mean()
print(f"v1: {v1}")
print(f"v2: {v2}")
print(f"1D 向量余弦相似度: {similarity_1d}")

v1: tensor([[[1., 2., 3.]]])
v2: tensor([[[1., 1., 3.]]])
1D 向量余弦相似度: 0.9669874906539917


In [12]:
import torch
a = torch.tensor([[1.,2.,3.]])

In [14]:
torch.argmax(a).item()

2

{'81': {'Prompt': [151644, 8948, 198, 2610, 525, 264, 10950, 11, 48050, 323, 10745, 17847, 13, 23240, 4226, 438, 1492, 3641, 438, 3204, 11, 1393, 1660, 6092, 13, 220, 4615, 11253, 1265, 537, 2924, 894, 27756, 11, 88635, 11, 24207, 11, 63782, 11, 20836, 11, 11406, 11, 476, 11816, 2213, 13, 5209, 5978, 429, 697, 14507, 525, 39318, 73215, 323, 6785, 304, 6993, 382, 2679, 264, 3405, 1558, 537, 1281, 894, 5530, 11, 476, 374, 537, 2097, 1832, 55787, 11, 10339, 3170, 4518, 315, 35764, 2494, 537, 4396, 13, 1416, 498, 1513, 944, 1414, 279, 4226, 311, 264, 3405, 11, 4486, 1513, 944, 4332, 895, 1995, 13, 151645, 198, 151644, 872, 198, 70492, 458, 22570, 5821, 5010, 1736, 911, 264, 3213, 8411, 311, 27521, 11, 38586, 12752, 11449, 323, 1969, 12, 4060, 38491, 13, 151645, 198, 151644, 77091, 198], 'Token': {'198': {'layer_index': [19, 20, 29, 30, 35, 36, 37, 39], 'similarity': 0.11242804676294327}, '32313': {'layer_index': [0, 1, 5, 8, 9, 11, 12, 13, 16, 21, 26, 28, 29, 30, 38, 39], 'similarity': 0.3